In [1]:
# using Python 3.11.9 for dependencies problems

In [2]:
# Example for hdfs usage, not used

# pip install hdfs
# ./hdfscli.cfg

#     [global]
#     default.alias = dev

#     [dev.alias]
#     url = http://localhost:9870

# from hdfs import Config

# client = Config().get_client('dev')
# test = client.list('/test')
# print(test)

# with client.read('/test/movies.csv') as reader:
#     movies = reader.read()

In [3]:
#!pip install findspark

In [1]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("Pagerank with graph Projections") \
        .master("yarn") \
        .config("spark.default.parallelism", "12") \
        .config("spark.executor.memory", "6g") \
        .config("spark.driver.memory", "1g") \
        .config("spark.hadoop.fs.DefaultFS", "hdfs://localhost:50054") \
        .config("spark.memory.storageFraction" , "0.3") \
        .config("spark.executor.memoryOverhead", "2g") \
        .config("spark.locality.wait.node", "0") \
        .getOrCreate()
# .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.1-s_2.12") \

In [ ]:
# Test the spark session
print(f"Spark Version: {spark.version}")

In [7]:
# ?

# from pyspark.sql import SQLContext
# sqlContext = SQLContext(spark)

In [ ]:
spark.sparkContext._conf.getAll()  # check the config

In [ ]:
hdfs_port = 50054

In [9]:
ratings = spark.read.csv('hdfs://localhost:'+ str(hdfs_port) +'/test/ratings.csv', header=True, inferSchema=True)

In [10]:
movies = spark.read.csv('hdfs://localhost:'+ str(hdfs_port) +'/test/movies.csv', header=True, inferSchema=True)

In [11]:
user_movie_matrix = ratings.join(movies, on="movieId", how="inner")

In [12]:
mapping_score = {
    0.5: -1.0,
    1: -1.0,
    1.5: -0.5,
    2: 0.0,
    2.5: 0.0,
    3: 0.0,
    3.5: 0.5,
    4: 1.0,
    4.5: 1.1,
    5: 1.2
}

In [13]:
from pyspark.sql.types import IntegerType, FloatType, StringType
from pyspark.sql.functions import col

# mapping_udf = udf(lambda x: mapping_score.get(x, 0))
# ratings = ratings.withColumn("mapped_rating", mapping_udf(col("rating")))


map_score_udf = spark.udf.register("map_score", lambda x: mapping_score.get(x, 0), FloatType())
user_movie_matrix = user_movie_matrix.withColumn("weight", map_score_udf(col("rating")))

In [14]:
# user_movie_edges = ratings.select("userId", "title", "mapped_rating")

edges = user_movie_matrix.select(
    col("userId").cast(StringType()).alias("src"), # <- why string?
    col("title").alias("dst"),
    col("weight")
)

In [15]:
from pyspark.sql.functions import lit

user_vertices = user_movie_matrix.select(col("userId").cast(StringType()).alias("id")).distinct().withColumn("bipartite", lit(0))
movie_vertices = user_movie_matrix.select(col("title").alias("id")).distinct().withColumn("bipartite", lit(1))
vertices = user_vertices.union(movie_vertices)

In [16]:
#!pip install graphframes

In [ ]:
# https://stackoverflow.com/questions/39261370/unable-to-run-a-basic-graphframes-example

# Depending on your spark version, all you have to do is download the graphframe jar corresponding to your version
# of spark here https://spark-packages.org/package/graphframes/graphframes.

# Then you'll have to copy the jar downloaded to your spark jar directory and rename it to graphframes.jar.

# # # pyspark --packages graphframes:graphframes:0.8.4-spark3.5-s_2.12
from graphframes import GraphFrame

user_movie_graph = GraphFrame(vertices, edges)

In [ ]:
import sys

def get_size(obj, seen=None):
    """Recursively find the size of objects."""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Mark as seen
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

# Assuming user_movie_graph is your object
memory_consumption = get_size(user_movie_graph)
print(f"Memory consumption of user_movie_graph: {memory_consumption} bytes")

In [ ]:
# using GraphX ?

In [ ]:
# Debug: Check the graph construction
print("First 10 nodes in the graph:")
user_movie_graph.vertices.show(10, truncate=False)

In [ ]:
print("First 10 edges in the graph:")
user_movie_graph.edges.show(10, truncate=False)

In [22]:
# Project movie-movie graph
movie_movie_edges = user_movie_graph.edges.alias("e1") \
    .join(user_movie_graph.edges.alias("e2"), col("e1.src") == col("e2.src")) \
    .select(
        col("e1.dst").alias("src"),
        col("e2.dst").alias("dst"),
        (col("e1.weight") + col("e2.weight")).alias("weight")
    ).filter(col("src") != col("dst"))

movie_movie_graph = GraphFrame(user_movie_graph.vertices, movie_movie_edges)

In [ ]:
# Debug: Check the graph construction
print("First 10 nodes in the graph:")
movie_movie_graph.vertices.show(10, truncate=False)
print("First 10 edges in the graph:")
movie_movie_graph.edges.show(10, truncate=False)

In [24]:
def create_preference_vector(user_id, user_movie_graph):
    edges = user_movie_graph.edges.filter(col("src") == user_id).rdd.map(lambda row: (row["dst"], row["weight"])).collect()
    print(f"Preference vector for user {user_id}: {list(edges)[:10]}")
    
    #
    tot = sum([weight for _, weight in edges])
    #
    
    print(f"Total weight for user {user_id}: {tot}")
    if tot > 0:
        return {movie: weight / tot for movie, weight in edges} # <- qua è diverso perchè assegno peso SOLO a quelli visti, in python normale assegnavmo a tutti i film qualcosa
    else:
        movies = user_movie_graph.vertices.filter(col("bipartite") == 1).select("id").rdd.map(lambda row: row[0]).collect()
        return {movie: 1 / len(movies) for movie in movies}

In [25]:
def predict_user(user_id, user_movie_graph, movie_movie_graph):
    p_vec = create_preference_vector(user_id, user_movie_graph)
    print(f"Preference vector for user {user_id}: {list(p_vec)[:10]}")
    already_seen = [movie for movie, weight in p_vec.items() if weight > 0]
    print(f"Already seen movies for user {user_id}: {list(already_seen)[:10]}")
    if len(already_seen) == len(p_vec):
        return []
    pagerank_results = movie_movie_graph.pageRank(resetProbability=0.95, maxIter=20, sourceId=p_vec.keys(), sourceWeight=p_vec.values()) # esiste sourceWeight? non sembra
    item_rank = pagerank_results.vertices.select("id", "pagerank").rdd.map(lambda row: (row["id"], row["pagerank"])).collectAsMap()
    recommendations = sorted(
        (movie for movie in item_rank if movie not in already_seen),
        key=lambda x: item_rank[x], reverse=True
    )
    return recommendations

In [ ]:
user = "10"
s_t = predict_user(user, user_movie_graph, movie_movie_graph)
print(f"Predicted movies for user {user}: {s_t[:10]}")